In [266]:
from datetime import datetime
import backtrader as bt
import backtrader.feeds as btfeeds
import yfinance as yf
import tulipy as ta
import matplotlib.pyplot as plt
import tiingo

In [285]:
class RSIfunc(bt.Strategy):

    def __init__(self):
        self.inds = dict()
        for i, d in enumerate(self.datas):

            self.inds[d] = dict()
            self.inds[d]['RSI']= bt.indicators.RelativeStrengthIndex()
            # self.inds[d]['stoch'] = (bt.indicators.Stochastic()).k
            self.inds[d]['ATR']= bt.indicators.AverageTrueRange(period=7)
            # self.inds[d]['MACD'] = bt.indicators.MACDHisto()

    def next(self):
        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size
            if not pos:  # no market / no orders
                if (self.inds[d]['RSI'][0] > 50):
                    
                    price = d.close[0]
                    price_limit = price + (2.8*self.inds[d]['ATR'][0])
                    price_stop = price - (1.2*self.inds[d]['ATR'][0])
                        
                    self.buy_bracket(
                        size=100,
                        price=price,
                        exectype=bt.Order.Market,
                        stopprice=price_stop,
                        limitprice=price_limit,
                        )

                elif (self.inds[d]['RSI'][0] < 50):

                    price = d.close[0]
                    price_limit = price - (2.8*self.inds[d]['ATR'][0])
                    price_stop = price + (1.2*self.inds[d]['ATR'][0])
                
                    self.sell_bracket(
                            size=100,
                            price=price,
                            exectype=bt.Order.Market,
                            stopprice=price_stop,
                            limitprice=price_limit,
                        )
            
            else:
                return

    def notify_trade(self, trade):
        dt = self.data.datetime.date()
        if trade.isclosed:
            print('{} {} Closed: PnL Gross {}, Net {}'.format(
                                                dt,
                                                trade.data._name,
                                                round(trade.pnl,2),
                                                round(trade.pnlcomm,2)))

In [287]:
class maCross(bt.Strategy):
    '''
    For an official backtrader blog on this topic please take a look at:

    https://www.backtrader.com/blog/posts/2017-04-09-multi-example/multi-example.html

    oneplot = Force all datas to plot on the same master.
    '''
    params = (
    ('sma1', 40),
    ('sma2', 200),
    ('oneplot', True)
    )

    def __init__(self):
        '''
        Create an dictionary of indicators so that we can dynamically add the
        indicators to the strategy using a loop. This mean the strategy will
        work with any numner of data feeds. 
        '''
        self.inds = dict()
        for i, d in enumerate(self.datas):
            self.inds[d] = dict()
            self.inds[d]['sma1'] = bt.indicators.SimpleMovingAverage(
                d.close, period=self.params.sma1)
            self.inds[d]['sma2'] = bt.indicators.SimpleMovingAverage(
                d.close, period=self.params.sma2)
            self.inds[d]['cross'] = bt.indicators.CrossOver(self.inds[d]['sma1'],self.inds[d]['sma2'])

            if i > 0: #Check we are not on the first loop of data feed:
                if self.p.oneplot == True:
                    d.plotinfo.plotmaster = self.datas[0]

    def next(self):
        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size
            if not pos:  # no market / no orders
                if self.inds[d]['cross'][0] == 1:
                    self.buy(data=d, size=1000)
                elif self.inds[d]['cross'][0] == -1:
                    self.sell(data=d, size=1000)
            else:
                if self.inds[d]['cross'][0] == 1:
                    self.close(data=d)
                    self.buy(data=d, size=1000)
                elif self.inds[d]['cross'][0] == -1:
                    self.close(data=d)
                    self.sell(data=d, size=1000)

    def notify_trade(self, trade):
        dt = self.data.datetime.date()
        if trade.isclosed:
            print('{} {} Closed: PnL Gross {}, Net {}'.format(
                                                dt,
                                                trade.data._name,
                                                round(trade.pnl,2),
                                                round(trade.pnlcomm,2)))


In [292]:
#Variable for our starting cash
startcash = 10000

#Create an instance of cerebro
cerebro = bt.Cerebro()

#create our data list
datalist = ['GE','NFLX','GOOG']

#Loop through the list adding to cerebro.
for i in range(len(datalist)):
    data = btfeeds.YahooFinanceData(dataname=datalist[i], fromdate= datetime(2019,1,1), todate= datetime(2020,6,1))
    cerebro.adddata(data, name=datalist[i])

# Set our desired cash start
cerebro.broker.setcash(startcash)

#Add our strategy
cerebro.addstrategy(maCross)

# Run over everything
cerebro.run()

#Get final portfolio Value
portvalue = cerebro.broker.getvalue()
pnl = portvalue - startcash

#Print out the final result
print('Final Portfolio Value: ${}'.format(portvalue))
print('P/L: ${}'.format(pnl))

cerebro.plot(iplot=False)
plt.savefig("bimb.png")

Final Portfolio Value: $-193540.0
P/L: $-203540.0


<IPython.core.display.Javascript object>

# BT (portfolio optimisation)

In [323]:
import bt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [324]:
from tiingo import TiingoClient

client = TiingoClient({'api_key' : '7fb8e25ac6335f846ae82d3f3a28ed535f313592'})

df = pd.read_csv(R"X:\GitHub\beatingENd\BeatingEnDUS\Own Screener\IBD50.csv")

# datalist = (df['Ticker'].to_list())
datalist = ['NFLX']

data = client.get_dataframe(datalist,
                                    metric_name='adjClose',
                                      frequency='daily',
                                      startDate='2015-02-02',
                                      endDate='2020-10-10')

In [325]:
tw = data.copy()

for i,d in enumerate(datalist):
    hap = tw[d].rolling(50).mean()
    fap = tw[d].rolling(200).mean()
    bap = (hap > fap).tolist()
    for z in range(len(tw)):
        if (np.isnan(fap[z])):
            tw[d][z] = 0 
        elif (bap[z]==True):
            tw[d][z] = 1/len(datalist)
        elif (bap[z]==False):
            tw[d][z] = -1/len(datalist)

In [326]:
tw

,NFLX
2015-02-02 00:00:00+00:00,0.0
2015-02-03 00:00:00+00:00,0.0
2015-02-04 00:00:00+00:00,0.0
2015-02-05 00:00:00+00:00,0.0
2015-02-06 00:00:00+00:00,0.0
...,...
2020-10-05 00:00:00+00:00,1.0
2020-10-06 00:00:00+00:00,1.0
2020-10-07 00:00:00+00:00,1.0
2020-10-08 00:00:00+00:00,1.0


In [233]:
tmp = bt.merge(tw, data)
ax = tmp.plot(figsize=(15,5), secondary_y=['tw'])
plt.savefig('fun.png')

In [328]:
crosses = bt.Strategy('ma_cross', [bt.algos.WeighTarget(tw),
                                    bt.algos.Rebalance()])


long_only = bt.Strategy('Benchmark', [bt.algos.RunOnce(),
                        bt.algos.SelectAll(),
                        bt.algos.WeighEqually(),
                        bt.algos.Rebalance()])

t = bt.Backtest(crosses,data)
s = bt.Backtest(long_only, data)

In [329]:
report = bt.run(t,s)
report.plot()
plt.title("Portfolio Returns")
plt.savefig("arg.png")

ma_cross
0% [############################# ] 100% | ETA: 00:00:00Benchmark
0% [############################# ] 100% | ETA: 00:00:00

<IPython.core.display.Javascript object>

In [330]:
report.set_riskfree_rate(0.001)
report.display()

Stat                 ma_cross    Benchmark
-------------------  ----------  -----------
Start                2015-02-01  2015-02-01
End                  2020-10-09  2020-10-09
Risk-free rate       0.10%       0.10%

Total Return         24.19%      756.10%
Daily Sharpe         0.29        1.10
Daily Sortino        0.48        1.97
CAGR                 3.88%       45.88%
Max Drawdown         -74.79%     -44.18%
Calmar Ratio         0.05        1.04

MTD                  7.88%       7.88%
3m                   6.24%       6.24%
6m                   45.50%      45.51%
YTD                  42.62%      66.71%
1Y                   15.35%      101.64%
3Y (ann.)            -1.06%      39.92%
5Y (ann.)            4.43%       36.60%
10Y (ann.)           3.88%       45.88%
Since Incep. (ann.)  3.88%       45.88%

Daily Sharpe         0.29        1.10
Daily Sortino        0.48        1.97
Daily Mean (ann.)    10.92%      46.53%
Daily Vol (ann.)     37.88%      42.09%
Daily Skew           0.61      

# Pyfolio

In [314]:
from datetime import datetime
import backtrader as bt
import backtrader.feeds as btfeeds
import yfinance as yf
import tulipy as ta
import pandas as pd
import matplotlib
import pyfolio as pf
import math

class RSIStrat(bt.Strategy):
    
    def __init__(self):
        self.dataclose= self.datas[0].close    # Keep a reference to the "close" line in the data[0] dataseries
        self.rsi= bt.indicators.RelativeStrengthIndex()
        self.stoch = bt.indicators.Stochastic()
        self.ATR= bt.indicators.AverageTrueRange(period=7)
        self.MAC = bt.indicators.MACDHisto()
        self.order = None # Property to keep track of pending orders.  There are no orders when the strategy is initialized.
        self.buyprice = None
        self.buycomm = None
    
    def log(self, txt, dt=None):
        # Logging function for the strategy.  'txt' is the statement and 'dt' can be used to specify a specific datetime
        dt = dt or self.datas[0].datetime.date(0)
        print('{0},{1}'.format(dt.isoformat(),txt))
    
    def notify_order(self, order):
        # 1. If order is submitted/accepted, do nothing 
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 2. If order is buy/sell executed, report price executed
        if order.status in [order.Completed]: 
            if order.isbuy():
                self.log('BUY EXECUTED, Price: {0:8.2f}, Cost: {1:8.2f}, Comm: {2:8.2f}'.format(
                    order.executed.price,
                    order.executed.value,
                    order.executed.comm))
                
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log('SELL EXECUTED, {0:8.2f}, Cost: {1:8.2f}, Comm{2:8.2f}'.format(
                    order.executed.price, 
                    order.executed.value,
                    order.executed.comm))
            
            self.bar_executed = len(self) #when was trade executed
        # 3. If order is canceled/margin/rejected, report order canceled
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            
        self.order = None
    
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        
        self.log('OPERATION PROFIT, GROSS {0:8.2f}, NET {1:8.2f}'.format(
            trade.pnl, trade.pnlcomm))
    
    def next(self):
        # Log the closing prices of the series from the reference
        # self.log('Close, {0:8.2f}'.format(self.dataclose[0]))

        if self.order: # check if order is pending, if so, then break out
            return
                
        # since there is no order pending, are we in the market?    
        if not self.position: # not in the market            
            if (self.rsi > 50) and (self.stoch.k > 50) and (self.MAC > 0):
                price = self.dataclose[0]
                price_limit = price + (2.8*self.ATR)
                price_stop = price - (1.2*self.ATR)
                atr_price = self.ATR[0]

                self.order = self.buy_bracket(
                    size=100,
                    price=price,
                    exectype=bt.Order.Market,
                    stopprice=price_stop,
                    limitprice=price_limit,
                )
                self.log('BUY AT: {:8.2f}, Limit: {:8.2f}, Stop: {:8.2f}'.format(price,price_limit,price_stop))

            elif (self.rsi < 50) and (self.MAC < 0) and (self.stoch < 50):
                price = self.dataclose[0]
                price_limit = price - (2.8*self.ATR)
                price_stop = price + (1.2*self.ATR)
                atr_price = self.ATR[0]

                self.order = self.sell_bracket(
                    size=100,
                    price=price,
                    exectype=bt.Order.Market,
                    stopprice=price_stop,
                    limitprice=price_limit,
                )
                self.log('SHORT AT: {:8.2f}, Limit: {:8.2f}, Stop: {:8.2f}'.format(price,price_limit,price_stop))

class RSIt(bt.Strategy):

    params = (('oneplot', True),)

    def __init__(self):
        self.inds = dict()
        self.o = dict()
        for i, d in enumerate(self.datas):
            self.inds[d] = dict()
            self.inds[d]['RSI']= bt.indicators.RelativeStrengthIndex()
            self.inds[d]['stoch'] = (bt.indicators.Stochastic()).k
            self.inds[d]['ATR']= bt.indicators.AverageTrueRange(period=7)
            self.inds[d]['MACD'] = bt.indicators.MACDHisto()

            if i > 0:
                if self.p.oneplot == True:
                    d.plotinfo.plotmaster = self.datas[0]

    def next(self):
        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size
            if not pos :  # no market / no orders
                if (self.inds[d]['RSI'][0] > 70) and (self.inds[d]['stoch'][0] > 50) and (self.inds[d]['MACD'][0] > 0):
                    # self.buy(data=d, size=100)                    
                    price = d.close[0]
                    price_limit = price + (1.5*self.inds[d]['ATR'][0])
                    price_stop = price - (1*self.inds[d]['ATR'][0])
                        
                    self.o[d] = self.buy_bracket(
                        data=d,
                        size=100,
                        price=price,
                        exectype=bt.Order.Market,
                        stopprice=price_stop,
                        limitprice=price_limit,
                        )

                elif (self.inds[d]['RSI'][0] < 30) and (self.inds[d]['stoch'][0] < 50) and (self.inds[d]['MACD'][0] < 0):
                    # self.sell(data=d, size=100)

                    price = d.close[0]
                    price_limit = price - (1.5*self.inds[d]['ATR'][0])
                    price_stop = price + (1*self.inds[d]['ATR'][0])
                
                    self.o[d] = self.sell_bracket(
                            data=d,
                            size=100,
                            price=price,
                            exectype=bt.Order.Market,
                            stopprice=price_stop,
                            limitprice=price_limit,
                        )
            
            elif pos:
                pass
            

    def notify_trade(self, trade):
        dt = self.data.datetime.date()
        if trade.isclosed:
            print('{} {} Closed: PnL Gross {}, Net {}'.format(
                                                dt,
                                                trade.data._name,
                                                round(trade.pnl,2),
                                                round(trade.pnlcomm,2)))

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.addstrategy(RSIt)
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.001)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta") #add analyzer

    #create our data list
    datalist = ['NFLX','FB','AAPL']

    #Loop through the list adding to cerebro.
    for i in range(len(datalist)):
        data = btfeeds.YahooFinanceData(dataname=datalist[i], fromdate= datetime(2015,1,1), todate= datetime(2020,6,1))
        cerebro.adddata(data, name=datalist[i])

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    strats = cerebro.run()
    firstStrat = strats[0]
    print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2016-01-29 FB Closed: PnL Gross -179.0, Net -200.62
2016-02-08 NFLX Closed: PnL Gross 1327.0, Net 1309.56
2016-02-10 NFLX Closed: PnL Gross -751.85, Net -768.95
2016-03-02 FB Closed: PnL Gross -980.83, Net -1001.89
2016-06-29 NFLX Closed: PnL Gross -226.0, Net -243.71
2016-06-29 FB Closed: PnL Gross -274.0, Net -296.4
2017-01-09 AAPL Closed: PnL Gross -648.7, Net -653.66
2018-08-06 NFLX Closed: PnL Gross -1799.62, Net -1867.73
2018-08-06 FB Closed: PnL Gross -1493.62, Net -1529.25
2018-08-21 NFLX Closed: PnL Gross -1342.68, Net -1407.83
2018-09-18 FB Closed: PnL Gross 1511.0, Net 1477.61
2019-08-05 NFLX Closed: PnL Gross 1772.74, Net 1709.84
2019-08-05 FB Closed: PnL Gross 2093.74, Net 2055.4
2019-10-07 NFLX Closed: PnL Gross -1414.76, Net -1468.68
2019-10-25 AAPL Closed: PnL Gross -1445.62, Net -1456.22
2019-10-31 FB Closed: PnL Gross -956.76, Net -995.31
Ending Portfolio Value: 94662.16


In [315]:
def printTradeAnalysis(analyzer):
    '''
    Function to print the Technical Analysis results in a nice format.
    '''
    #Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = (total_won / total_closed) * 100
    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed,total_won,total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

In [318]:
printTradeAnalysis(firstStrat.analyzers.ta.get_analysis())

Trade Analysis Results:
               Total Open     Total Closed   Total Won      Total Lost     
               0              16             4              12             
               Strike Rate    Win Streak     Losing Streak  PnL Net        
               25.0           3              8              -5337.84       


In [319]:
analy = firstStrat.analyzers.ta.get_analysis()

In [321]:
analy

AutoOrderedDict([('total',
                  AutoOrderedDict([('total', 16),
                                   ('open', 0),
                                   ('closed', 16)])),
                 ('streak',
                  AutoOrderedDict([('won',
                                    AutoOrderedDict([('current', 0),
                                                     ('longest', 3)])),
                                   ('lost',
                                    AutoOrderedDict([('current', 3),
                                                     ('longest', 8)]))])),
                 ('pnl',
                  AutoOrderedDict([('gross',
                                    AutoOrderedDict([('total',
                                                      -4808.972560354447),
                                                     ('average',
                                                      -300.5607850221529)])),
                                   ('net',
                           